In [ ]:
################################################################################
#
# Script to run an obstacle avoidance example for the TwoPlayerUnicycle4D.
#
################################################################################

import os
import numpy as np
import matplotlib.pyplot as plt

from obstacle_dist_cost import ObstacleDistCost
from car_5d import Car5D
from product_multiplayer_dynamical_system import \
    ProductMultiPlayerDynamicalSystem
from two_player_unicycle_4d import TwoPlayerUnicycle4D
from ilq_solver_reach_avoid import ILQSolver            #Uncomment this (this is for the one-player one obstacle case)
#from ilq_solver_mult_obs_try import ILQSolver            #Uncomment this (this is for the one-player multiple obstacle case)
from point import Point
from proximity_cost import ProximityCost
from obstacle_cost import ObstacleCost
from semiquadratic_cost import SemiquadraticCost
from quadratic_cost import QuadraticCost
#from player_cost import PlayerCost
from player_cost_reach_avoid import PlayerCost
from box_constraint import BoxConstraint
from visualizer import Visualizer
from logger import Logger
from semiquadratic_polyline_cost import SemiquadraticPolylineCost
from quadratic_polyline_cost import QuadraticPolylineCost
import sys
import signal

# General parameters.
TIME_HORIZON = 2.0   # s  #Change back to 2
TIME_RESOLUTION = 0.1 # s
HORIZON_STEPS = int(TIME_HORIZON / TIME_RESOLUTION)
LOG_DIRECTORY = "./logs/two_player_zero_sum/"
MAX_V = 7.0 # m/s

# Create dynamics.
car1 = Car5D(4.0)
dynamics = ProductMultiPlayerDynamicalSystem(
    [car1], T=TIME_RESOLUTION)

# Choose an initial state and control laws.
car1_theta0 = np.pi / 2.2 # 90 degree heading # change back to 2.2!
car1_v0 = 2.0             # 5 m/s initial speed
car1_x0 = np.array([
    [6.5],
    [0.0],
    [car1_theta0],     #Originally 6.5, 0.0
    [0.0],
    [car1_v0]
])


# theta0 = np.pi / 10
# v0 = 5.0

# theta0 = np.pi / 6
# v0 = 5.0

x0 = car1_x0

P1s = [np.zeros((car1._u_dim, dynamics._x_dim))] * HORIZON_STEPS
alpha1s = [np.zeros((car1._u_dim, 1))] * HORIZON_STEPS

# Create the example environment. It will have a couple of circular obstacles
# laid out like this:
#                           x goal
#
#                      ()
#               ()
#                            ()
#
#          x start
goal = Point(6.5, 35.0)
#goal_radius = 2.0

obstacle_centers = [Point(6.5, 15.0)]
obstacle_radii = [4.5]                             #Uncomment this (this is for one-player one obstacle)!!!!!
car1_position_indices_in_product_state = (0, 1)

#obstacle_centers = [Point(6.5, 15.0), Point(0.0, 20.0), Point(16.0, 20.0)]
#obstacle_radii = [4.5, 1.5, 4.0]                                            #Uncomment this (this is for the one-player multiple obstacle case)!!!!!
#car1_position_indices_in_product_state = (0, 1)

goal_cost = ProximityCost(car1_position_indices_in_product_state,
                          point=goal,
                          max_distance=np.inf,
                          apply_after_time=HORIZON_STEPS - 1,
                          name="goal")
#obstacle_costs = [ObstacleCost(
#    position_indices=(0, 1), point=p, max_distance=r,     #Uncomment this (this is for the one player multiple obstacle case)
#    name="obstacle_%f_%f" % (p.x, p.y))
#                  for p, r in zip(obstacle_centers, obstacle_radii)]

#for n in range(N):
    
obstacle_costs = [ObstacleDistCost(
    position_indices=(0, 1), point=p, max_distance=r,    #Uncomment this (this is for the one-player one obstacle case)
    name="obstacle_%f_%f" % (p.x, p.y))
                  for p, r in zip(obstacle_centers, obstacle_radii)]

w_cost = QuadraticCost(dimension=0, origin=0, name="w_cost")
a_cost = QuadraticCost(dimension=1, origin=0, name="a_cost")

#dvx_cost = QuadraticCost(dimension=0, origin=0, name="dvx_cost")
#dvy_cost = QuadraticCost(dimension=1, origin=0, name="dvy_cost")

# Add light quadratic around original values for theta/v.
v_cost_upper = SemiquadraticCost(
    dimension=3, threshold=MAX_V, oriented_right=True, name="v_cost_upper")
v_cost_lower = SemiquadraticCost(
    dimension=3, threshold=0, oriented_right=False, name="v_cost_lower")

# Polyline
#player1_position_indices_in_product_state = (0, 1)
#car1_polyline = Polyline([Point(6.5, 0.0), Point(6.5, 40.0)])
#car1_polyline_boundary_cost = SemiquadraticPolylineCost(
#    car1_polyline, 1.0, car1_position_indices_in_product_state,
#    "car1_polyline_boundary")
#car1_polyline_cost = QuadraticPolylineCost(
#    car1_polyline, car1_position_indices_in_product_state, "car1_polyline")

OBSTACLE_WEIGHT = 200.0 # HJI: 100
GOAL_WEIGHT = 50.0 # HJI: 400  #Change back to 100
#D_WEIGHT = 1000.0 # HJI: 1000
U_WEIGHT = 10.0 # HJI: 100

# V_WEIGHT = 100.0
V_WEIGHT = 0.1 # HJI: 50

# Build up total costs for both players. This is basically a zero-sum game.
player1_cost = PlayerCost()
player1_cost.add_cost(goal_cost, "x", GOAL_WEIGHT)
#for cost in obstacle_costs:
#    player1_cost.add_cost(cost, "x", OBSTACLE_WEIGHT)

#player1_cost.add_cost(v_cost_upper, "x", V_WEIGHT)
#player1_cost.add_cost(v_cost_lower, "x", V_WEIGHT)
#player1_cost.add_cost(w_cost, 0, U_WEIGHT)
#player1_cost.add_cost(a_cost, 0, U_WEIGHT)

# Visualizer.
visualizer = Visualizer(
    [(0, 1)],
    [goal_cost] + obstacle_costs,
    [".-b"],
    1,
    False,
    plot_lims=[-11, 100, 0, 100])

# Logger.
if not os.path.exists(LOG_DIRECTORY):
    os.makedirs(LOG_DIRECTORY)

path_to_logfile = os.path.join(LOG_DIRECTORY, "unicycle_4d_example.pkl")
if len(sys.argv) > 1:
    path_to_logfile = os.path.join(LOG_DIRECTORY, sys.argv[1])

print("Saving log file to {}...".format(path_to_logfile))
logger = Logger(path_to_logfile)

# Set up ILQSolver.
solver = ILQSolver(dynamics,
                   [player1_cost],
                   x0,
                   [P1s],
                   [alpha1s],
                   1.0,
                   None,
                   logger,
                   visualizer,
                   None,
                  [car1_position_indices_in_product_state, goal, obstacle_centers, obstacle_radii])

def handle_sigint(sig, frame):
    print("SIGINT caught! Saving log data...")
    logger.dump()
    print("...done, exiting!")
    sys.exit(0)

signal.signal(signal.SIGINT, handle_sigint)

solver.run()


#ProximityCost(car1_position_indices_in_product_state, goal, max_distance=np.inf, apply_after_time=HORIZON_STEPS - 1, name="goal")


#ObstacleDistCost(car1_position_indices_in_product_state, point=p, max_distance=r, name="obstacle_%f_%f" % (p.x, p.y)) for p, r in zip(obstacle_centers, obstacle_radii)


#[car1_position_indices_in_product_state, goal, obstacle_centers, obstacle_radii]

In [ ]:
a = []
a.append(0)
a.append(1)
a.append(6)
print(a)

# Weight for each: 20
# Initial step-size: 0.03

In [ ]:
import numpy as np

np.argmin((2,3))

In [ ]:
from collections import deque

a = deque()
for ii in range(5):
    a.appendleft("True")

a[2] = False
print(a[2])